In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly import graph_objects as go
from emutools.tex import DummyTexDoc, StandardTexDoc
import arviz as az
from autumn.infrastructure.remote import springboard
from inputs.constants import SUPPLEMENT_PATH, RUNS_PATH, RUN_IDS, PRIMARY_ANALYSIS, BURN_IN, PLOT_START_DATE, ANALYSIS_END_DATE
from aust_covid.calibration import get_targets
from emutools.calibration import plot_spaghetti, plot_param_hover_spaghetti, get_target_from_name
from aust_covid.plotting import plot_cdr_examples, plot_subvariant_props, plot_3d_spaghetti
from aust_covid.utils import add_image_to_doc
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'spaghetti', 'Example calibration runs', 'austcovid')
targets = get_targets(app_doc)
spaghetti = pd.read_hdf(RUNS_PATH / RUN_IDS['mob'] / 'output/results.hdf', 'spaghetti')

In [ ]:
key_indicators_fig = plot_spaghetti(spaghetti, ['notifications_ma', 'deaths_ma', 'adult_seropos_prop', 'reproduction_number'], 2, targets)
key_outputs_title = 'Key outputs for randomly sampled runs from calibration algorithm.'
add_image_to_doc(key_indicators_fig.update_layout(showlegend=False), 'multioutput_spaghetti', 'svg', key_outputs_title, app_doc, 'Calibration results')
key_indicators_fig

In [ ]:
idata = az.from_netcdf(RUNS_PATH / RUN_IDS['mob'] / 'output/calib_full_out.nc')
burnt_idata = idata.sel(draw=np.s_[BURN_IN:])
n_samples = 12
sampled_idata = az.extract(burnt_idata, num_samples=n_samples)
cdr_caption = 'Examples of the modelled effect of various starting CDR proportion parameters.'
print(cdr_caption)
cdr_fig = plot_cdr_examples(sampled_idata.variables['start_cdr'])
caption = f'Modelled case detection ratio over time for {str(n_samples)} randomly selected parameter draws from calibration algorithm.'
add_image_to_doc(cdr_fig, 'cdr_examples', 'svg', cdr_caption, app_doc, 'Model construction', caption=caption)
cdr_fig

In [ ]:
plot_param_hover_spaghetti(spaghetti['notifications_ma'], idata)

In [ ]:
app_doc.write_doc()

In [ ]:
plot_3d_spaghetti('adult_seropos_prop', spaghetti, targets)